# Similarity Per Parameter set Calculations

In [1]:
import glob
import shutil
import pandas as pd
from dap import DAPcython
from dap.utils import obs_params, load_current
from tqdm import tqdm
from scipy.spatial import distance

In [ ]:
import os
os.chdir('../')

In [2]:
dt = 1e-2
params, labels = obs_params(reduced_model=False)
data_dir = '/home/alteska/Desktop/LFI_DAP/data/rawData/2015_08_26b.dat'

In [3]:
# load the file
directory = './parameters/'
dir = glob.glob(directory + '*')

In [4]:
fname_start = dir[0].find('dap_')
fname_stop = dir[0].find('n_')
fname = dir[0][fname_start:fname_stop] + '.csv'

df_param = pd.read_csv(fname)

In [5]:
# calculate DAP
# load the input data
Ir, vr, tr, t_onr, t_offr, dtr = load_current(data_dir, protocol='rampIV', ramp_amp=3.1)
Is, vs, ts, t_ons, t_offs, dts = load_current(data_dir, protocol='IV', ramp_amp=1)

# define a model
dap = DAPcython(-75, params)

In [6]:
# run models on original parameters
U_step = dap.simulate(dts, ts, Is)
U_ramp = dap.simulate(dtr, tr, Ir)

In [13]:
# calculate the similarities
d_step = distance.euclidean(vs, U_step)
d_step

2699.36325187886

In [12]:
d_ramp = distance.euclidean(vr, U_ramp)
d_ramp

503.354576791061

In [14]:
d_step+d_ramp

3202.717828669921

In [8]:
# run for all cells and save into the the DF
df_paramT = df_param.transpose()
df_paramT.head()

,0,1,2,3
Unnamed: 0,gbar_nap,gbar_leak,gbar_nat,gbar_kdr
1x14,17.1737,1.07467,131.423,3.41051
1x35,17.0972,1.03987,117.684,3.73748
1x9,19.0074,0.991425,95.3657,2.97607
1x19,16.3705,1.0285,141.775,3.43003


In [9]:
df_paramT.drop('Unnamed: 0', inplace=True)
df_paramT.head()

,0,1,2,3
1x14,17.1737,1.07467,131.423,3.41051
1x35,17.0972,1.03987,117.684,3.73748
1x9,19.0074,0.991425,95.3657,2.97607
1x19,16.3705,1.0285,141.775,3.43003
1x24,15.1533,1.10018,163.721,3.05554


In [10]:
for i, j in tqdm(df_paramT.iterrows()):
    # get parameters
    par_temp = j.values

    print(par_temp)

34it [00:00, 3593.37it/s]

[17.17373165830238 1.0746656390459317 131.42294160134227 3.410513693315566]
[17.097218725893402 1.0398685282458358 117.68422996045706
 3.737484012824088]
[19.007379625846454 0.9914247812585802 95.36573095222616 2.976070246356468]
[16.370542944176396 1.0285044203791625 141.77457317459502
 3.4300320685043664]
[15.15333474426479 1.1001843946128176 163.72109414916417 3.055540649117644]
[16.51589092744879 1.1217549093033687 120.13933653900406 3.414984528678401]
[14.387061714904222 1.0938739837995843 164.14991852028754
 3.1281664057193974]
[16.183387753341865 1.0358684546896884 108.91678074546587
 2.9942777815645263]
[16.586714040113748 1.0469091967844906 169.62574312814826 3.17433147230534]
[18.934190084574553 1.0256818301902488 115.46528640453613
 3.1945525210692467]
[19.11305080821854 1.1346109550143262 53.6393994945139 3.220647187680138]
[16.884771042925188 1.0720373765927735 158.74480350537937
 3.113928039541624]
[16.041616357273462 1.0329686846283668 117.14503040978228
 3.5262406599973

In [11]:
for i, j in tqdm(df_param.iterrows()):
    # get parameters
    par_temp = j.values

    print(par_temp)

    # define a model
    dap = DAPcython(-75, j)

    # run model
    U_step_x = dap.simulate(dts, ts, Is)
    U_ramp_x = dap.simulate(dtr, tr, Ir)

    # calculate distance for both currents
    dis_step = distance.euclidean(vs, U_step_x)
    dis_ramp = distance.euclidean(vr, U_ramp_x)

    # save into new columns
    df_paramT.loc[i, 'distance_ramp'] = dis_ramp
    df_paramT.loc[i, 'distance_step'] = dis_step
    df_paramT.loc[i, 'distance_sum'] = dis_ramp + dis_step

# save the new data DataFrame
df_paramT.to_csv(fname + '_similarity.csv')

# remove the temp directory
shutil.rmtree(directory)

0it [00:00, ?it/s]

['gbar_nap' 17.17373165830238 17.097218725893402 19.007379625846454
 16.370542944176396 15.15333474426479 16.51589092744879 14.387061714904222
 16.183387753341865 16.586714040113748 18.934190084574553
 19.11305080821854 16.884771042925188 16.041616357273462
 13.530289884851722 17.11043949343459 16.543604426909994
 17.138812020545682 18.91970690796745 16.42225786699909 16.90263865514906
 16.586808406774033 19.076895584426815 16.547376118880987
 18.434825006590387 17.68756034723681 16.339341348801977
 15.024634711590526 16.749955472158266 15.36648929508807
 18.050893372186323 19.543175040738586 16.077840910203115 16.4120636141781
 16.126094574384314]


ValueError: You can only provide 1, 2, 3, 4, 5 or 11 parameters!